In [ ]:
import os
import glob
from PIL import Image
import numpy as np
import torch
import torch.optim as optim
from torchvision.transforms import Resize,ToTensor,Pad
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from DanbooruTagger import DanbooruTagger
import torch.utils.data as data
import torch

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
print(device)


In [ ]:
#该cell主要是从当前文件夹下获取图片和标签数据，标签数据是预先定义好的，保存在tokens.txt文件中。也可以传入别的名称调用其他文件作为参数，可以考虑使用TokensSplit.py。
#由于数据集的选择，图片位于imagefolder文件夹下，txt文本在txtfolder下，图片和标签通过文件名一一对应，txt中标签使用 ', '分割。自行根据需要修改该部分方法
#出于显存和内存的考虑，使用分批次加载，每批次加载1024张到内存，在train函数中判断该批次是否完成训练后在手动重置
# 从本地文件夹中加载图片和文本数据
image_folder = "pixiv\pixiv_top50_deepdan\pixiv_top50_fin"
txt_folder = "pixiv\pixiv_top50_moat"
transf = ToTensor()#这个函数方法是将PIL图像转换为PyTorch张量，并将其归一化到[0, 1]的范围内。

class dataLoader:
    def __init__(self, image_folder ,txt_folder,labels_file="tokens.txt"):
        self.image_folder = image_folder
        self.txt_folder = txt_folder
        self.image_files = glob.glob(os.path.join(image_folder, "*.jpg"))
        self.num=0
        self.resize=Resize((512,512))#该处定义了图片的尺寸，可以根据需要修改，建议小一些，在6g显卡运存下，256*256 每批可以到16，大约1分钟1024张，如果512*512，每批只能到4张，要10分钟才有1024张
        self.all_labels=[]
        self.end=False
        with open(labels_file, "r") as f:
            self.all_labels = f.read().strip().split("\n")
    
    def numclasses(self):
        return len(self.all_labels)
    def len(self):
        return len(self.image_files)
    def isEnd(self):
        return self.end
    def getXY(self):
        X=[]
        Y=[]
        for nums in range(self.num,self.len() if self.num+1024>self.len() else self.num+1024):
            image_file = self.image_files[nums]

            #获取图片对应的标签文本
            txt_file = os.path.join(txt_folder, os.path.basename(image_file).replace(".jpg", ".txt"))
            with open(txt_file, "r") as f:
                text = f.read().strip().split(", ")

            #读取图片
            img = Image.open(image_file)
            if img.mode != 'RGB':
                img = img.convert('RGB')#将图片转换为RGB模式，避免图片为单通道等问题

            #填充图片边缘，将短边填充到和长边长度一致
            maxlen=img.width if img.width > img.height else img.height
            img=Pad([int((maxlen-img.width)/2),int((maxlen-img.height)/2)],fill=(0,0,0),padding_mode='constant')(img)

            X.append(self.resize(img))#将图片尺寸调整一致，有利于模型训练，否则会出现图片尺寸不一致导致每次只单张地训练，而且模型学习的深度不够
            Y.append(text)

        self.num+=1024
        self.end=True if self.num>=self.len() else False
        
        # 将字符串组转换为索引值
        label_indices = [[self.all_labels.index(label) if label in self.all_labels else -1 for label in label_list] for label_list in Y]
        target = torch.full((len(label_indices), self.numclasses()), 0)

        for i, indices in enumerate(label_indices):
            if indices != -1:
                target[i][indices] = .8 #出于模型的最后一层的激活函数的选择问题，在设置为1的情况下，模型只学到输出值的分布，认为是梯度消失之类的问题
        return X,target
    
loader=dataLoader(image_folder,txt_folder)
print(loader.len())
print(loader.numclasses())

'''激活函数的选择问题'''
'''
关于模型最后一层的分类层的激活函数，在选择relu下，loss值几乎不收敛，认为模型几乎没学到东西，在选择sigmoid的条件下，模型只学到输出值的分布，任何图像都输出一样的结果
softmax不适合多标签分类模型，tanh和sigmoid的问题基本一致
最后选择了sigmoid，将结果放在了0.8附近，在这个位置，sigmoid的导数值较大，模型才可以正常训练并且分类
我只测试了一次，即正面标签为0.8，反面标签为0，成功实现
但是不确定是否为偶然现象，或者别的参数，如0.85和0.05的数据会否更好
'''

In [ ]:
# 训练函数的定义，传入参数依次为 模型，loss函数，优化器，epoch数，数据加载器，每批数据大小，模型名称
# 每训练一轮，保存一次模型
# 每训练loader的一个小包数据，输出一次loss，每轮训练输出一次总loss

def train_model(model, criterion, optimizer, num_epochs,loader,batch=16,model_name='efftagImg3'):

    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        while not loader.isEnd():#该批次的数据是否读完
            X,Y=loader.getXY()
            run_loss=0.0
            i=0

            while i+batch < len(X) :  #取一个batch的数据

                #将图片拼接成一个完整的inputs的tensor
                size=[1,3,X[i].width,X[i].height]
                inputs=transf(X[i]).resize_(size)
                for w in range(1,batch):
                    inputs=torch.cat((inputs,transf(X[i+w]).resize_(size)),0)
                inputs=inputs.to(device)

                #将标签拼接成一个完整的labels的tensor
                labels=torch.Tensor(Y[i:i+batch]).to(device)

                outputs = model(inputs)  # 前向传播
                outputs=torch.where(torch.isnan(outputs),torch.zeros_like(outputs),outputs)#将非法输出转化为0，早期训练时长出现输出为nan的情况，建议不删
                
                optimizer.zero_grad()
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                run_loss += loss.item()
                del inputs
                del outputs
                i+=batch
                torch.cuda.empty_cache()
                
            print(f'Epoch {epoch+1}/{num_epochs} when {loader.num}, Loss: {run_loss/(len(X)/batch)}')
            del X
            del Y
            
        loader.num=0
        loader.end=False
        torch.save(model,model_name+str(epoch)+'.pt')
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/(loader.len()/batch)}')

In [ ]:
#采用迁移训练，模型通过创建是冻结除最后的分类层以外的参数，参照DanbooruTagger.py内的描述
model = DanbooruTagger(loader.numclasses())
model.to(device)

num_epochs = 10  # 可自行设置训练的轮数
criterion =  nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=.02)  
train_model(model, criterion, optimizer, 1,loader,64)
for param in model.parameters():
    param.requires_grad = True
optimizer = optim.Adam(model.parameters(), lr=.002)  
train_model(model, criterion, optimizer, num_epochs,loader,16)

#torch.save(model,'tagImg3_ls.pt')#最后保存一次模型，有利于之后找到模型，前面的可以选择保存在指定文件夹下，这个放在项目下